In [1]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [7]:
df=pd.read_csv("ilac_tahmin.csv")
df.head()

,Yas,Cinsiyet,Tansiyon,Kolestrol,SoPo_Oran,İlaç
0,23,K,YÜKSEK,YÜKSEK,25.355,ilacY
1,47,E,DÜŞÜK,YÜKSEK,13.093,ilacC
2,47,E,DÜŞÜK,YÜKSEK,10.114,ilacC
3,28,K,NORMAL,YÜKSEK,7.798,ilacX
4,61,K,DÜŞÜK,YÜKSEK,18.043,ilacY
